In [5]:
#imports 

import tensorflow as tf
import numpy as np
import matplotlib as matplotlib
import matplotlib.ticker as ticker
from urllib.request import urlretrieve
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import spacy
import sys
import os
import csv

In [9]:
#load GloVe vectors

GloVe_vectors_file = "glove.6B.50d.txt"

glove_wordmap = {}
with open(GloVe_vectors_file, "r", encoding ="utf8") as glove:
    for line in glove:
        name, vector = tuple(line.split(" ", 1))
        glove_wordmap[name] = np.fromstring(vector, sep=" ")


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [11]:
#load spacy & tqdm
tqdm_installed = True
nlp = spacy.load('en')

In [12]:
#turns sentence into (n,d) matrix, 
#n - no. of tokens
#d - no. of dimensions each vector word has
#also lemmatizes sentence before converting into vector form

def sentence2sequence(sentence):
    tokens = nlp(sentence.lower())
    rows = []
    words = []
    #Greedy search for tokens
    for token in tokens:
        word = token.lemma_
        if word in glove_wordmap:
            rows.append(glove_wordmap[word])
            words.append(word)
    return rows, words

In [13]:
#turn labels into score that could be fed into a network
#POSITIVE = [1.0 0.0 0.0]
#NEUTRAL = [0.0 1.0 0.0]
#CONTRADICTION = [0.0 0.0 1.0]

def score_setup(row):
    convert_dict = {
      'ENTAILMENT': 0,
      'NEUTRAL': 1,
      'CONTRADICTION': 2
    }
    score = np.zeros((3,))
    if row in convert_dict: score[convert_dict[row]] += 1
    return score / (1.0*np.sum(score))

In [14]:
#fits the different sentence shapes

def fit_to_size(matrix, shape):
    res = np.zeros(shape)
    slices = [slice(0,min(dim,shape[e])) for e, dim in enumerate(matrix.shape)]
    res[slices] = matrix[slices]
    return res

In [15]:
#loads training data

def process_training_data():
    with open("SICK_train.txt","r") as data:
        train = csv.DictReader(data, delimiter='\t')
        hyp_sentences = []
        evi_sentences = []
        scores = []
        labels = []
        for row in train:
            hyp_sentences.append(np.vstack(
                    sentence2sequence(row["sentence_A"])[0]))
            evi_sentences.append(np.vstack(
                    sentence2sequence(row["sentence_B"])[0]))
            scores.append(score_setup(row["entailment_judgment"]))

        hyp_sentences = np.stack([fit_to_size(x, (max_hypothesis_length, vector_size))
                          for x in hyp_sentences])
        evi_sentences = np.stack([fit_to_size(x, (max_evidence_length, vector_size))
                          for x in evi_sentences])
        return (hyp_sentences, evi_sentences), np.array(scores)

In [16]:
# RNN vars
max_hypothesis_length, max_evidence_length = 30, 30
batch_size, vector_size, hidden_size = 128, 50, 64

lstm_size = hidden_size

training_iterations_count = 100000

display_step = 10

data_feature_list, correct_scores = process_training_data()

l_h, l_e = max_hypothesis_length, max_evidence_length
N, D, H = batch_size, vector_size, hidden_size
l_seq = l_h + l_e

c:\users\alets\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


In [17]:
weight_decay = 0.0001

learning_rate = 0.0005

input_p, output_p = 0.5, 0.5

tf.reset_default_graph()

lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

lstm_drop =  tf.contrib.rnn.DropoutWrapper(lstm, input_p, output_p)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [18]:
#N - number of elements in each batch
#l_h - maximum length of a hypothesis sentence
#l_e - maximum length of an evidence sentence 
#D - size of used GloVe vectors 

hyp = tf.placeholder(tf.float32, [N, l_h, D], 'hypothesis')
evi = tf.placeholder(tf.float32, [N, l_e, D], 'evidence')
y = tf.placeholder(tf.float32, [N, 3], 'label')

In [19]:
#LSTM used to look backwards through the senteneces 
lstm_back = tf.contrib.rnn.BasicLSTMCell(lstm_size)

#dropout wrapper for lstm_back
lstm_drop_back = tf.contrib.rnn.DropoutWrapper(lstm_back, input_p, output_p)


In [20]:
# randomly initialize weights

fc_initializer = tf.random_normal_initializer(stddev=0.1)

# storage for the weights.
# multiplied by two because there are two LSTMs
fc_weight = tf.get_variable('fc_weight', [2*hidden_size, 3],
                            initializer = fc_initializer)

# storage for the bias
fc_bias = tf.get_variable('bias', [3])

In [21]:
# regularization on weights

tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES,
                     tf.nn.l2_loss(fc_weight))

# shaping the inputs so it can data can be fed through the net

x = tf.concat([hyp, evi], 1) # N, (Lh+Le), d
# Permuting batch_size and n_steps
x = tf.transpose(x, [1, 0, 2]) # (Le+Lh), N, d
# Reshaping to (n_steps*batch_size, n_input)
x = tf.reshape(x, [-1, vector_size]) # (Le+Lh)*N, d
# Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
x = tf.split(x, l_seq,)

In [22]:
# runs the input through two recurrent nets, one forward, one backward
# and combines the outputs

rnn_outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm, lstm_back,
                                                            x, dtype=tf.float32)

In [23]:
#scores are relative certainties for output matching an entailment value

classification_scores = tf.matmul(rnn_outputs[-1], fc_weight) + fc_bias


In [24]:
with tf.variable_scope('Accuracy'):
    predicts = tf.cast(tf.argmax(classification_scores, 1), 'int32')
    y_label = tf.cast(tf.argmax(y, 1), 'int32')
    corrects = tf.equal(predicts, y_label)
    num_corrects = tf.reduce_sum(tf.cast(corrects, tf.float32))
    accuracy = tf.reduce_mean(tf.cast(corrects, tf.float32))

with tf.variable_scope("loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
        logits = classification_scores, labels = y)
    loss = tf.reduce_mean(cross_entropy)
    total_loss = loss + weight_decay * tf.add_n(
        tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))

optimizer = tf.train.AdamOptimizer(learning_rate)

opt_op = optimizer.minimize(total_loss)

init = tf.global_variables_initializer()

# Launch the Tensorflow session
sess = tf.Session()
sess.run(init)

In [25]:
training_epochs = 7

for i in range(training_epochs):
    print("EPOCH ", i)

    training_iterations = range(0,training_iterations_count,batch_size)
    if tqdm_installed:
        # Add a progress bar if TQDM is installed
        training_iterations = tqdm(training_iterations)

    for i in training_iterations:

        # Select indices for a random data subset
        batch = np.random.randint(data_feature_list[0].shape[0], size=batch_size)
        #y_scores = np.reshape(correct_score,(len(correct_values),1))
        # Use the selected subset indices to initialize the graph's
        #   placeholder values
        hyps, evis, ys = (data_feature_list[0][batch,:],
                          data_feature_list[1][batch,:],
                          correct_scores[batch])
    # Run the optimization with these initialized values
        sess.run([opt_op], feed_dict={hyp: hyps, evi: evis, y: ys})
    # display_step: how often the accuracy and loss should
    #   be tested and displayed.
        if (i/batch_size) % display_step == 0:
        # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={hyp: hyps, evi: evis, y: ys})
        # Calculate batch loss
            tmp_loss = sess.run(loss, feed_dict={hyp: hyps, evi: evis, y: ys})
        # Display results
            print("Iter " + str(i/batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(tmp_loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))


EPOCH  0


  0%|                                                                                          | 0/782 [00:00<?, ?it/s]

Iter 0.0, Minibatch Loss= 1.028856, Training Accuracy= 0.55469


  1%|▊                                                                                 | 8/782 [00:03<23:06,  1.79s/it]

Iter 10.0, Minibatch Loss= 0.977827, Training Accuracy= 0.55469


...

 98%|██████████████████████████████████████████████████████████████████████████████▌ | 768/782 [00:27<00:00, 27.91it/s]

Iter 770.0, Minibatch Loss= 0.093306, Training Accuracy= 0.96875


 99%|███████████████████████████████████████████████████████████████████████████████▌| 778/782 [00:28<00:00, 27.30it/s]

Iter 780.0, Minibatch Loss= 0.042524, Training Accuracy= 0.99219


100%|████████████████████████████████████████████████████████████████████████████████| 782/782 [00:28<00:00, 27.52it/s]


In [26]:
evidences = ["Two spectators are kickboxing and some people are watching"]

hypotheses = ["Two people are kickboxing and spectators are watching"]

sentence1 = [fit_to_size(np.vstack(sentence2sequence(evidence)[0]),
                         (30, 50)) for evidence in evidences]

sentence2 = [fit_to_size(np.vstack(sentence2sequence(hypothesis)[0]),
                         (30,50)) for hypothesis in hypotheses]

prediction = sess.run(classification_scores, feed_dict={hyp: (sentence1 * N),
                                                        evi: (sentence2 * N),
                                                        y:[[0,0,0]]*N})
print(["Positive", "Neutral", "Negative"][np.argmax(prediction[0])]+
      " entailment")
print("prediction",prediction[0])

c:\users\alets\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


Neutral entailment
prediction [-0.04078504  0.34014255 -0.4341808 ]


In [27]:
evidences = ["There is no boy playing outdoors and there is no man smiling"]

hypotheses = ["A group of kids is playing in a yard and an old man is standing in the background"]

sentence1 = [fit_to_size(np.vstack(sentence2sequence(evidence)[0]),
                         (30, 50)) for evidence in evidences]

sentence2 = [fit_to_size(np.vstack(sentence2sequence(hypothesis)[0]),
                         (30,50)) for hypothesis in hypotheses]

prediction = sess.run(classification_scores, feed_dict={hyp: (sentence1 * N),
                                                        evi: (sentence2 * N),
                                                        y:[[0,0,0]]*N})
print(["Positive", "Neutral", "Negative"][np.argmax(prediction[0])]+
      " entailment")
print("prediction",prediction[0])

evidences = ["A man is performing a trick on a green bicycle"]

hypotheses = ["A man dressed in black is riding a bike"]

sentence1 = [fit_to_size(np.vstack(sentence2sequence(evidence)[0]),
                         (30, 50)) for evidence in evidences]

sentence2 = [fit_to_size(np.vstack(sentence2sequence(hypothesis)[0]),
                         (30,50)) for hypothesis in hypotheses]

prediction = sess.run(classification_scores, feed_dict={hyp: (sentence1 * N),
                                                        evi: (sentence2 * N),
                                                        y:[[0,0,0]]*N})
print(["Positive", "Neutral", "Negative"][np.argmax(prediction[0])]+
      " entailment")
print("prediction",prediction[0])

evidences = ["A man is performing a trick on a green bicycle"]

hypotheses = ["A man is not performing a trick on a green bicyle"]

sentence1 = [fit_to_size(np.vstack(sentence2sequence(evidence)[0]),
                         (30, 50)) for evidence in evidences]

sentence2 = [fit_to_size(np.vstack(sentence2sequence(hypothesis)[0]),
                         (30,50)) for hypothesis in hypotheses]

prediction = sess.run(classification_scores, feed_dict={hyp: (sentence1 * N),
                                                        evi: (sentence2 * N),
                                                        y:[[0,0,0]]*N})
print(["Positive", "Neutral", "Negative"][np.argmax(prediction[0])]+
      " entailment")
print("prediction",prediction[0])

c:\users\alets\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


Neutral entailment
prediction [-2.3046298  5.136812  -3.564967 ]
Neutral entailment
prediction [-0.41946328  0.98137367 -1.1349845 ]
Positive entailment
prediction [ 0.7671622  -0.7415304  -0.78941536]


In [28]:
def process_test_data():
    with open("SICK_test.txt","r") as data:
        train = csv.DictReader(data, delimiter='\t')
        hyp_sentences = []
        evi_sentences = []
        for row in train:
            hyp_sentences.append(row["sentence_A"])
            evi_sentences.append(row["sentence_B"])

        return (hyp_sentences, evi_sentences)

hyp_test, evi_test = process_test_data()

In [29]:
#generate predictions on the whole test set
results = []

for i in range(len(hyp_test)):
    evidences = []
    hypotheses = []
    evidences.append(evi_test[i])
    hypotheses.append(hyp_test[i])
    
    sentence1 = [fit_to_size(np.vstack(sentence2sequence(evidence)[0]),
                         (30, 50)) for evidence in evidences]

    sentence2 = [fit_to_size(np.vstack(sentence2sequence(hypothesis)[0]),
                         (30,50)) for hypothesis in hypotheses]
    
    prediction = sess.run(classification_scores, feed_dict={hyp: (sentence1 * N),
                                                        evi: (sentence2 * N),
                                                        y:[[0,0,0]]*N})

    results.append(score_setup(["ENTAILMENT", "NEUTRAL", "CONTRADICTION"][np.argmax(prediction[0])]))

    

c:\users\alets\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


In [30]:
#get test correct labels
t_label = []
with open("SICK_test_annotated.txt","r") as data:
    train = csv.DictReader(data, delimiter='\t')
    for row in train:
        t_label.append(score_setup(row["entailment_judgment"]))

test_labels = np.array(t_label)

In [31]:
# calculate accuracy of predictions
results = np.array(results)
print(results)
print(test_labels.shape)
print(results[0])
correct = 0

for i in range(results.shape[0]):
    if(np.array_equal(results[i],test_labels[i])):
        correct = correct + 1
acc = correct / results.shape[0]

print("ACCURACY IS ", acc * 100, "%");


[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
(4927, 3)
[0. 1. 0.]
ACCURACY IS  60.604830525674856 %


In [ ]:
sess.close()